# Projeto Persona ILUMEO

## Objetivos

* Ajudar gestores de marketing selecionarem quais celebridades devem ser contratadas para campanhas publicitárias;

## Cliente: Vivara

* Quer uma celebridade para substituir a Gisele Bundchen
* Manter a marca forte
* Expandir negócio além da classe AB
* Alguém muito famosa

## Público

* Mulheres de Meia Idade (Maduras)
* Mulheres ousadas
* Mulheres modernas

## Dados

* Pesquisa **online** sobre a imagem de cada celebridade
* Número de seguidores nas redes sociais (Não utilizado)

## Desafio

* (x) Definir critério de decisão
* (x) Descrever racíocinio
* ( ) Montar um dashboard

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [203]:
from scipy import stats

In [250]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Colunas com nomes mais legíveis
columns = [
    "Celebridade",
    "Ent_Idade",
    "Ent_Civil",
    "Ent_Filhos",
    "Ent_Renda",
    "Ent_Genero",
    "Cel_Conhece",
    "Cel_Gosta",        # Identificacao com a celebridade
    "Cel_Identifico",   # Identificacao com a celebridade
    "Cel_Respeito",     # Identificacao com a celebridade
    "Cel_Atencao",      # Identificacao com a celebridade
    "Cel_Confio",       # Identificacao com a celebridade
    "Cel_BomNoQueFaz",  # Visao positiva e sofisticada da celebridade
    "Cel_Inteligente",  # Visao positiva e sofisticada da celebridade
    "Cel_Determinado",  # Visao positiva e sofisticada da celebridade
    "Cel_Honesto",      # Visao positiva e sofisticada da celebridade
    "Cel_Engracado",    # Visao positiva e sofisticada da celebridade
    "Cel_Lindo",        # Visao positiva e sofisticada da celebridade
    "Cel_Sedutor",      # Visao positiva e sofisticada da celebridade
    "Cel_Povo",         # Apelo social
    "Cel_CaraBrasil",   # Apelo social
    "Cel_Inovador",     # Visao positiva e sofisticada da celebridade
    "Cel_Desafio",      # Visao positiva e sofisticada da celebridade      
    "Cel_TenhoVisto",         # Apelo comercial
    "Cel_PerfilPropagandas",  # Apelo comercial
    "Cel_EuCompraria",        # Apelo comercial
    "Cel_ProjetosSociais",         # Apelo social
    "Cel_EuMeEnvolveriaSocial",    # Apelo social
    "Cel_EuIndicariaProdutos",     # Apelo social
    "Cel_EuComentariaSocial"       # Apelo social
]

In [79]:
mar2019 = pd.read_excel('./Base_Persona_V2.xlsx', na_filter=False, sheetname="Março 2019", header=1, names=columns)
jul2019 = pd.read_excel('./Base_Persona_V2.xlsx', na_filter=False, sheetname="Julho 2019", header=1, names=columns)
dez2019 = pd.read_excel('./Base_Persona_V2.xlsx', na_filter=False, sheetname="Dezembro 2019", header=1, names=columns)

mar2019.shape, jul2019.shape, dez2019.shape

((4138, 30), (3400, 30), (3610, 30))

**Tamanho das amostras é similar e a cardinalidade igual. Como a pesquisa é online, irei assumir que as amostras foram obtidas de forma aleatória e representam, com certeza, menos de 10% da população, condição para que o C.L.T. seja válido e possa aplicar testes de hipóteses e intervalos de confiança**

# Celebridades

In [80]:
# Resolvendo inconsistências nos nomes das celebridades

mar2019.Celebridade = mar2019.Celebridade.str.strip()
jul2019.Celebridade = jul2019.Celebridade.str.strip()
dez2019.Celebridade = dez2019.Celebridade.str.strip()

In [81]:
mar2019_count = mar2019["Celebridade"].value_counts()
jul2019_count = jul2019["Celebridade"].value_counts()
dez2019_count = dez2019["Celebridade"].value_counts()

pd.concat([mar2019_count, jul2019_count, dez2019_count],keys=["Marco", "Julho", "Dezembro"], axis=1)

,Marco,Julho,Dezembro
Ana,240.0,200.0,200.0
Beatriz,235.0,200.0,200.0
Bruna,244.0,200.0,200.0
Carol,253.0,200.0,200.0
Clara,210.0,200.0,232.0
Fátima,344.0,200.0,221.0
Gilda,244.0,200.0,202.0
Giovana,234.0,200.0,200.0
Iris,226.0,200.0,205.0
Juliana,234.0,200.0,211.0


**Vou remover o Luan Santana pois ele não se enquadra no perfil e há dados faltando nas duas primeiras pesquisas.**

In [82]:
dez2019[dez2019.Celebridade == "Letícia"].shape

(0, 30)

In [83]:
removidos = [
    "Luan Santana",
    "Letícia"
];

**Não há dados da Letícia na pesquisa de dezembro, provavelmente porque deixaram de conhecê-la?. Como não tenho acesso ao porque disso acontecer, nem conselho de um especialista no assunto, também irei eliminá-la.**

In [84]:
dados = pd.concat([mar2019, jul2019, dez2019], axis=0, keys=["Marco", "Julho", "Dezembro"])
dados.shape

(11148, 30)

In [85]:
dados = dados[~dados.Celebridade.isin(removidos)]
dados.shape

(10505, 30)

In [86]:
dados.Celebridade.value_counts()

Fátima     765
Marina     701
Luciana    689
Sabrina    660
Carol      653
Maria      650
Gilda      646
Juliana    645
Bruna      644
Clara      642
Luana      640
Ana        640
Beatriz    635
Giovana    634
Iris       631
Telma      630
Name: Celebridade, dtype: int64

# Pergunta 1: Celebridade Conhecida?

**Verificar se no agregado das pesquisa a celebridade é conhecida durante o ano todo, pois como a Gisele Bundchen tem uma imagem pública há anos, é importante que a substituta tenha o mesmo perfil.**

In [93]:
dados.Cel_Conhece.value_counts()

1    9231
2    1274
Name: Cel_Conhece, dtype: int64

In [124]:
conhece = dados[dados.Cel_Conhece == 2]["Celebridade"].value_counts() \
             .rename_axis('Celebridade').reset_index(name='counts')
conhece

,Celebridade,counts
0,Ana,317
1,Giovana,211
2,Marina,140
3,Luana,96
4,Telma,71
5,Carol,65
6,Beatriz,56
7,Bruna,54
8,Gilda,54
9,Sabrina,45


In [133]:
df = conhece['counts']
conhece['z_score'] = (df - df.mean()) / df.std()
conhece[conhece['z_score'] > 1]

,Celebridade,counts,z_score
0,Ana,317,2.942257
1,Giovana,211,1.628390


**Removi as celebridades com base no z-score que ela apresenta das demais, onde as celebridades com um desvio padrão acima da média foram removida por ser considerada "desconhecida acima da média"**

In [134]:
removidos_conhece = [
    "Ana", "Giovana"
]

In [135]:
dados = dados[~dados.Celebridade.isin(removidos_conhece)]
dados.shape

(9231, 30)

# Entrevistados

**Tentar entender melhor a demografia dos entrevistados e como posso usá-los para estratificar os dados**

## Idade

A idade do entrevistado deve refletir, a princípio, a faixa etária do público alvo da Vivara. Que, segundo o teste, busca mulheres de meia-idade.

https://veja.abril.com.br/tecnologia/meia-idade-comeca-aos-35-e-termina-aos-58-diz-estudo/

In [136]:
dados["Ent_Idade"].describe()

count    9231.000000
mean       31.597877
std        11.703521
min         3.000000
25%        24.000000
50%        32.000000
75%        39.000000
max        57.000000
Name: Ent_Idade, dtype: float64

## Gênero

* 1	Masculino
* 2	Feminino

In [139]:
dados["Ent_Genero"].value_counts()

1    5453
2    3778
Name: Ent_Genero, dtype: int64

## Estado Civil

* 1	Solteiro
* 2	Casado

In [138]:
dados["Ent_Civil"].value_counts()

1    4814
2    4031
3     343
4      43
Name: Ent_Civil, dtype: int64

## Tem Filhos?

* 1	Não
* 2	Sim

In [141]:
dados["Ent_Filhos"].value_counts()

2    4862
1    4369
Name: Ent_Filhos, dtype: int64

## Renda

    1	Classe Econômica A/B
	2	
	3	
	4	
	5	Classe Econômica C
	6	
	7	
	8	

In [140]:
dados["Ent_Renda"].value_counts()

5    2568
4    2145
3    1763
6    1248
2     874
1     269
7     235
8     129
Name: Ent_Renda, dtype: int64

# Definir Grupos

O primeiro grupo será mais direto e representa o público alvo: as mulheres de meia-idade.

In [151]:
grupo_alvo_a = dados[(dados["Ent_Genero"] == 2) & (dados["Ent_Idade"] >= 35)]

dados.shape, grupo_alvo_a.shape 

((9231, 30), (1561, 30))

In [152]:
grupo_alvo_a.Celebridade.value_counts()

Marina     124
Maria      122
Carol      122
Iris       117
Beatriz    116
Telma      113
Sabrina    113
Fátima     113
Luciana    113
Luana      111
Clara      106
Gilda      104
Juliana     99
Bruna       88
Name: Celebridade, dtype: int64

**mulheres** O grupo será formado pelas mulheres da pesquisa sem distinção de idade

In [149]:
mulheres = dados[(dados["Ent_Genero"] == 2)]

dados.shape, mulheres.shape

((9231, 30), (3778, 30))

In [150]:
mulheres.Celebridade.value_counts()

Fátima     300
Marina     298
Carol      289
Maria      282
Luciana    282
Sabrina    276
Juliana    269
Telma      269
Clara      260
Iris       256
Gilda      255
Luana      252
Beatriz    250
Bruna      240
Name: Celebridade, dtype: int64

## Hipótese 1:  Há diferença entre os dois grupos. Mulheres na meia-idade conhecem celebridades diferentes do restante. Ou seja, a diferença em proporção se dá devido a faixa etária ou apenas chance?

In [179]:
conhece = grupo_alvo_a[grupo_alvo_a.Cel_Conhece == 1].Celebridade.value_counts()
n_conhece = grupo_alvo_a[grupo_alvo_a.Cel_Conhece == 2].Celebridade.value_counts()

In [180]:
grupo_alvo_a_celeb = grupo_alvo_a.Celebridade.value_counts()
grupo_alvo_a_conhece = pd.concat([conhece, n_conhece, grupo_alvo_a_celeb], keys=["Conhece", "Nao Conhece", "Total"], axis=1)

In [181]:
grupo_alvo_a_conhece

,Conhece,Nao Conhece,Total
Beatriz,102,14,116
Bruna,80,8,88
Carol,114,8,122
Clara,102,4,106
Fátima,108,5,113
Gilda,98,6,104
Iris,109,8,117
Juliana,93,6,99
Luana,96,15,111
Luciana,112,1,113


In [166]:
conhece = mulheres[mulheres.Cel_Conhece == 1].Celebridade.value_counts()
n_conhece = mulheres[mulheres.Cel_Conhece == 2].Celebridade.value_counts()
total = mulheres.Celebridade.value_counts()


mulheres_conhece = pd.concat([conhece, n_conhece, total], keys=["Conhece", "Nao Conhece", "Total"], axis=1)

mulheres_conhece

,Conhece,Nao Conhece,Total
Beatriz,221,29,250
Bruna,208,32,240
Carol,248,41,289
Clara,240,20,260
Fátima,279,21,300
Gilda,224,31,255
Iris,239,17,256
Juliana,252,17,269
Luana,209,43,252
Luciana,271,11,282


In [171]:
p_grupo_alvo_a = grupo_alvo_a_conhece.Conhece / grupo_alvo_a_conhece.Total

In [172]:
p_mulheres = mulheres_conhece.Conhece / mulheres_conhece.Total

In [176]:
df = pd.concat([p_grupo_alvo_a, p_mulheres, grupo_alvo_a_conhece.Total, mulheres_conhece.Total], axis=1, keys=["p_a", "p_m", "n_a", "n_m"])
df

,p_a,p_m,n_a,n_m
Beatriz,0.879310,0.884000,116,250
Bruna,0.909091,0.866667,88,240
Carol,0.934426,0.858131,122,289
Clara,0.962264,0.923077,106,260
Fátima,0.955752,0.930000,113,300
Gilda,0.942308,0.878431,104,255
Iris,0.931624,0.933594,117,256
Juliana,0.939394,0.936803,99,269
Luana,0.864865,0.829365,111,252
Luciana,0.991150,0.960993,113,282


**Teste de Hipótese entre duas proporções de grupos distindos usando a Normal Distribution**

* Ho: p_a - p_m = 0
* Ha: p_a - p_m != 0

**Framework**

A significância escolhida foi 10% pois o teste foi feito online e não há muita segurança sobre as condiçoes de independencia, assim diminuindo o poder do teste mas já validando a hipótese central do problema. 

* 1. Pooled Proportion

p_pool = (p1 x n1 + p2 x n2) / (n1 x n2)

* 2. Std Error

std = sqrt{ p_pool x ( 1 - p_pool ) x [ (1/n1) + (1/n2) ] }

* 3. Z-score

z = (p1 - p2) / std

* 4. p-value

P(>z) < 0.10 -> Há diferença entre os grupos

In [191]:
df["p_pool"] = (df.p_a * df.n_a + df.p_m * df.n_m) / (df.n_a + df.n_m)

In [198]:
df["SE"] = np.sqrt((df.p_pool * (1 - df.p_pool)) * ((1/df.n_a) + (1/df.n_m)))

In [204]:
df["z"] = (df.p_a - df.p_m) / df.SE

In [211]:
df["p-value"] = stats.norm.sf(abs(df.z)) * 2
df

,p_a,p_m,n_a,n_m,p_pool,std,SE,z,p-value
Beatriz,0.879310,0.884000,116,250,0.882514,0.036174,0.036174,-0.129642,0.896850
Bruna,0.909091,0.866667,88,240,0.878049,0.040780,0.040780,1.040331,0.298186
Carol,0.934426,0.858131,122,289,0.880779,0.034987,0.034987,2.180680,0.029207
Clara,0.962264,0.923077,106,260,0.934426,0.028526,0.028526,1.373745,0.169521
Fátima,0.955752,0.930000,113,300,0.937046,0.026808,0.026808,0.960610,0.336748
Gilda,0.942308,0.878431,104,255,0.896936,0.035375,0.035375,1.805698,0.070966
Iris,0.931624,0.933594,117,256,0.932976,0.027906,0.027906,-0.070588,0.943725
Juliana,0.939394,0.936803,99,269,0.937500,0.028455,0.028455,0.091055,0.927449
Luana,0.864865,0.829365,111,252,0.840220,0.041740,0.041740,0.850505,0.395044
Luciana,0.991150,0.960993,113,282,0.969620,0.019109,0.019109,1.578222,0.114515


In [213]:
df[df["p-value"] < 0.10]

,p_a,p_m,n_a,n_m,p_pool,std,SE,z,p-value
Carol,0.934426,0.858131,122,289,0.880779,0.034987,0.034987,2.180680,0.029207
Gilda,0.942308,0.878431,104,255,0.896936,0.035375,0.035375,1.805698,0.070966


## Resultado: Reconhecimento no Público Meia-Idade


Apenas as celebridades Carol e Gilda são mais conhecidas entre as mulheres de meia-idade segundo os resultados do teste de diferença de proporção.

| Celebridade | Meia-Idade | Mulheres no Geral |
|-------------|------------|-------------------|
| Carol | 93.4% | 85.8% |
| Gilda | 94.2% | 87.8% |


Vou guardar esses resultados por enquanto. Agora vou investigar o reconhecimento entre as classes sociais.

## Classe Social

In [214]:
dados["Ent_Renda"].value_counts()

5    2568
4    2145
3    1763
6    1248
2     874
1     269
7     235
8     129
Name: Ent_Renda, dtype: int64

In [219]:
dados["Ent_ClsSocial"] = dados["Ent_Renda"].apply(lambda x: "AB" if x <= 4 else "C+")

In [220]:
dados.Ent_ClsSocial.value_counts()

AB    5051
C+    4180
Name: Ent_ClsSocial, dtype: int64

**Os dados estão relativamente balanceados (17% de diferença) entre as classes então posso tratar como grupos discriminativos a seguir**

In [244]:
ab_total = dados[dados.Ent_ClsSocial == "AB"].Celebridade.value_counts()
c_total = dados[dados.Ent_ClsSocial == "C+"].Celebridade.value_counts()

pd.concat([ab_total, c_total], axis=1, keys=["AB", "C+"])

,AB,C+
Beatriz,355,280
Bruna,359,285
Carol,339,314
Clara,359,283
Fátima,416,349
Gilda,348,298
Iris,355,276
Juliana,346,299
Luana,337,303
Luciana,381,308


In [245]:
ab_total = dados[(dados.Ent_ClsSocial == "AB") & (dados.Cel_Conhece == 1)].Celebridade.value_counts()
c_total = dados[(dados.Ent_ClsSocial == "C+") & (dados.Cel_Conhece == 1)].Celebridade.value_counts()

pd.concat([ab_total, c_total], axis=1, keys=["AB", "C+"])

,AB,C+
Beatriz,325,254
Bruna,324,266
Carol,304,284
Clara,342,273
Fátima,395,336
Gilda,310,282
Iris,330,270
Juliana,329,276
Luana,284,260
Luciana,371,301


In [246]:
cls_c.shape, cls_ab.shape

((3878, 31), (4607, 31))

**Não há uma discrepância muito grande entre a contagem de entrevistados que conhecem as celebridades entre as classes, o que nos possibilita fazer testes comparativos entre as amostras a seguir:**

In [235]:
cls_c = dados[(dados.Ent_ClsSocial == "C+") & (dados.Cel_Conhece == 1)]
cls_ab = dados[(dados.Ent_ClsSocial == "AB") & (dados.Cel_Conhece == 1)]

# Identificação com a celebridade
### Variável agregada

**A seguir defini as features que, segundo meu critério (especialista seria o ideal), estão relacionadas com a identificação entre o entrevistado e a celebridade.**

In [243]:
features_identificacao = [
    "Cel_Gosta",
    "Cel_Identifico",
    "Cel_Respeito",
    "Cel_Atencao",
    "Cel_Confio"
]

In [269]:
cls_c["Calc_Ident"] = 0
cls_ab["Calc_Ident"] = 0

for feature in features_identificacao:
    cls_c["Calc_Ident"] += cls_c[feature]
    cls_ab["Calc_Ident"] += cls_ab[feature]

In [272]:
cls_c["Calc_Ident"] = pd.to_numeric(cls_c["Calc_Ident"])
cls_ab["Calc_Ident"] = pd.to_numeric(cls_ab["Calc_Ident"])

ab_mu = cls_ab[["Celebridade", "Calc_Ident"]].groupby("Celebridade").mean()
c_mu = cls_c[["Celebridade", "Calc_Ident"]].groupby("Celebridade").mean()

ab_std = cls_ab[["Celebridade", "Calc_Ident"]].groupby("Celebridade").std()
c_std = cls_c[["Celebridade", "Calc_Ident"]].groupby("Celebridade").std()

ab_n = cls_ab[["Celebridade", "Calc_Ident"]].groupby("Celebridade").count()
c_n = cls_c[["Celebridade", "Calc_Ident"]].groupby("Celebridade").count()

df = pd.concat([ab_mu, ab_std, ab_n, c_mu, c_std, c_n], axis=1, 
               keys=["ab_mu", "ab_std", "ab_n", "c_mu", "c_std", "c_n"])
df

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n
,Calc_Ident,Calc_Ident,Calc_Ident,Calc_Ident,Calc_Ident,Calc_Ident
Celebridade,,,,,,
Beatriz,20.095385,9.264635,325,20.515748,10.096655,254
Bruna,23.864198,8.211603,324,24.263158,8.871744,266
Carol,23.157895,8.729227,304,22.785211,8.623638,284
Clara,23.339181,8.869972,342,23.183150,8.877608,273
Fátima,24.093671,8.706951,395,24.392857,8.874270,336
Gilda,26.616129,7.631759,310,26.226950,8.334526,282
Iris,27.366667,7.923386,330,27.237037,8.064063,270
Juliana,26.386018,8.311756,329,26.420290,7.302614,276


## Hipótese 2: Há diferença de indentificação entre as celebridades e as classes sociais AB e C+?

**Teste de Hipótese entre as médias das classes AB e C+**

* Ho: mu_ab - mu_c+ = 0
* Ha: mu_ab - mu_c+ != 0

O objetivo é selecionar celebridades que não possuem diferença de indentificação, ou seja, o p-value > 0.10

**Framework**

1 Calcular os degrees of freedom para distribuição t

df = n - 1

2 Calcultar a t-statistic para cada celebridade

https://en.m.wikipedia.org/wiki/T-statistic

3 Calcular o p-value através de stats.t.sf

4 Rejeiar ou não a hipótese usando uma significância de 0.10

In [314]:
# degrees of freedom
df["df"] = df[["ab_n", "c_n"]].min(axis=1)
df["df"] = df["df"] - 1

In [312]:
df["t_stat"] = (df.ab_mu - df.c_mu) / ((df.ab_std**2 / df.ab_n) + (df.c_std**2 / df.c_n))

In [316]:
df["p_value"] = stats.t.sf(np.abs(df.t_stat), df["df"])*2

In [328]:
df[df["p_value"] < 0.10]

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n,SE,df,t_stat,p_value
,Calc_Ident,Calc_Ident,Calc_Ident,Calc_Ident,Calc_Ident,Calc_Ident,,,,
Celebridade,,,,,,,,,,
Marina,25.341935,7.82903,310,24.454183,8.335040,251,0.688844,250,1.870898,0.062527
Telma,27.407767,7.75280,309,26.552000,8.146703,250,0.678228,249,1.860393,0.064009


## Resultado: Identificação entre classes sociais

Verifica-se que a maioria dos entrevistados se identificam com as celebridades apesar da classe econômica. Exceto no caso da Marina e Telma, onde a diferença da média de identificação é estatísticamente significante para dintinguir os dois grupos. Ou seja, uma pessoa da classe AB pode se identificar com uma delas, enquanto que uma da C+ não.

| Celebridade | Classe AB | Classe C+ | Diferença entre as médias |
|-------------|-----------|-----------|---------------------------|
| Marina | 25.3 | 24.45 | 0.85 |
| Telma | 27.4 | 26.55 | 0.8 |

## Visão Positiva e Sofisticada da Celebridade
### Variável agregada

**A seguir defini as features que, segundo meu critério (especialista seria o ideal), estão relacionadas com uma visão positiva e sofisticada do entrevistado em relação a celebridade.**

In [329]:
features_visao_positiva = [
    "Cel_BomNoQueFaz",
    "Cel_Inteligente",
    "Cel_Determinado",
    "Cel_Honesto",
    "Cel_Engracado",
    "Cel_Lindo",
    "Cel_Sedutor",
    "Cel_Inovador",
    "Cel_Desafio" 
]

In [330]:
cls_c["Calc_Visao"] = 0
cls_ab["Calc_Visao"] = 0

for feature in features_visao_positiva:
    cls_c["Calc_Visao"] += cls_c[feature]
    cls_ab["Calc_Visao"] += cls_ab[feature]

In [331]:
cls_c["Calc_Visao"] = pd.to_numeric(cls_c["Calc_Visao"])
cls_ab["Calc_Visao"] = pd.to_numeric(cls_ab["Calc_Visao"])

ab_mu = cls_ab[["Celebridade", "Calc_Visao"]].groupby("Celebridade").mean()
c_mu = cls_c[["Celebridade", "Calc_Visao"]].groupby("Celebridade").mean()

ab_std = cls_ab[["Celebridade", "Calc_Visao"]].groupby("Celebridade").std()
c_std = cls_c[["Celebridade", "Calc_Visao"]].groupby("Celebridade").std()

ab_n = cls_ab[["Celebridade", "Calc_Visao"]].groupby("Celebridade").count()
c_n = cls_c[["Celebridade", "Calc_Visao"]].groupby("Celebridade").count()

df = pd.concat([ab_mu, ab_std, ab_n, c_mu, c_std, c_n], axis=1, 
               keys=["ab_mu", "ab_std", "ab_n", "c_mu", "c_std", "c_n"])
df

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n
,Calc_Visao,Calc_Visao,Calc_Visao,Calc_Visao,Calc_Visao,Calc_Visao
Celebridade,,,,,,
Beatriz,45.963077,14.684076,325,45.582677,15.962377,254
Bruna,47.768519,12.270385,324,47.609023,13.154716,266
Carol,48.657895,12.194575,304,45.989437,13.062904,284
Clara,46.087719,14.914553,342,45.512821,13.945035,273
Fátima,47.227848,13.109215,395,46.491071,13.731756,336
Gilda,52.641935,10.229290,310,51.262411,11.372781,282
Iris,53.503030,11.562503,330,53.129630,11.535699,270
Juliana,51.066869,12.518831,329,49.898551,11.717766,276


## Hipótese 3: Há diferença de percepção de positividade e sofisticação entre as celebridades e as classes sociais AB e C+?

**Teste de Hipótese entre as médias das classes AB e C+**

* Ho: mu_ab - mu_c+ = 0
* Ha: mu_ab - mu_c+ != 0

O objetivo é selecionar celebridades que não possuem diferença de visão, ou seja, o p-value > 0.10

**Framework**

1 Calcular os degrees of freedom para distribuição t

df = n - 1

2 Calcultar a t-statistic para cada celebridade

https://en.m.wikipedia.org/wiki/T-statistic

3 Calcular o p-value através de stats.t.sf

4 Rejeiar ou não a hipótese usando uma significância de 0.10

In [332]:
# degrees of freedom
df["df"] = df[["ab_n", "c_n"]].min(axis=1)
df["df"] = df["df"] - 1

In [333]:
df["t_stat"] = (df.ab_mu - df.c_mu) / ((df.ab_std**2 / df.ab_n) + (df.c_std**2 / df.c_n))

In [334]:
df["p_value"] = stats.t.sf(np.abs(df.t_stat), df["df"])*2

In [336]:
df[df["p_value"] <= 0.10]

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n,df,t_stat,p_value
,Calc_Visao,Calc_Visao,Calc_Visao,Calc_Visao,Calc_Visao,Calc_Visao,,,
Celebridade,,,,,,,,,
Carol,48.657895,12.194575,304,45.989437,13.062904,284,283,2.448097,0.014969
Gilda,52.641935,10.229290,310,51.262411,11.372781,282,281,1.732644,0.084256
Maria,39.017341,15.234771,346,36.336806,15.507706,288,287,1.780099,0.076118


### Resultado: Visão Positiva e Sofisticada entre as Classes

Verifica-se que a maioria dos entrevistados possuem uma visão similar das celebridades apesar da classe econômica. Exceto as celebridades Carol, Gilda e Maria, onde a diferença entre as médias de identificação é estatísticamente significante para dintinguir os dois grupos. Ou seja, uma pessoa da classe AB pode se identificar com uma delas, enquanto que uma da C+ não.

Nota-se tambem que a média da Maria é baixa para ambas as classes, o que é preocupante de acordo com os objetivos do cliente.

# Apelo Social
### Variável agregada

In [337]:
features_apelo_social = [
    "Cel_Povo",
    "Cel_CaraBrasil",
    "Cel_ProjetosSociais",
    "Cel_EuMeEnvolveriaSocial",
    "Cel_EuIndicariaProdutos",
    "Cel_EuComentariaSocial"
]

In [338]:
cls_c["Calc_Social"] = 0
cls_ab["Calc_Social"] = 0

for feature in features_apelo_social:
    cls_c["Calc_Social"] += cls_c[feature]
    cls_ab["Calc_Social"] += cls_ab[feature]

cls_c["Calc_Social"] = pd.to_numeric(cls_c["Calc_Social"])
cls_ab["Calc_Social"] = pd.to_numeric(cls_ab["Calc_Social"])

ab_mu = cls_ab[["Celebridade", "Calc_Social"]].groupby("Celebridade").mean()
c_mu = cls_c[["Celebridade", "Calc_Social"]].groupby("Celebridade").mean()

ab_std = cls_ab[["Celebridade", "Calc_Social"]].groupby("Celebridade").std()
c_std = cls_c[["Celebridade", "Calc_Social"]].groupby("Celebridade").std()

ab_n = cls_ab[["Celebridade", "Calc_Social"]].groupby("Celebridade").count()
c_n = cls_c[["Celebridade", "Calc_Social"]].groupby("Celebridade").count()

df = pd.concat([ab_mu, ab_std, ab_n, c_mu, c_std, c_n], axis=1, 
               keys=["ab_mu", "ab_std", "ab_n", "c_mu", "c_std", "c_n"])
df

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n
,Calc_Social,Calc_Social,Calc_Social,Calc_Social,Calc_Social,Calc_Social
Celebridade,,,,,,
Beatriz,26.969231,11.232901,325,26.818898,11.797647,254
Bruna,29.790123,9.511233,324,28.951128,10.671840,266
Carol,29.006579,10.192858,304,27.510563,10.520545,284
Clara,29.701754,10.693762,342,28.692308,10.550326,273
Fátima,29.301266,10.673649,395,29.125000,10.656474,336
Gilda,32.422581,8.681648,310,30.886525,10.186931,282
Iris,34.690909,8.554221,330,34.318519,8.372509,270
Juliana,33.455927,9.239630,329,32.833333,8.340118,276


## Hipótese 4: Há diferença de apelo social entre as celebridades e as classes sociais AB e C+?

**Teste de Hipótese entre as médias das classes AB e C+**

* Ho: mu_ab - mu_c+ = 0
* Ha: mu_ab - mu_c+ != 0

O objetivo é selecionar celebridades que não possuem diferença de visão, ou seja, o p-value > 0.10

**Framework**

1 Calcular os degrees of freedom para distribuição t

df = n - 1

2 Calcultar a t-statistic para cada celebridade

https://en.m.wikipedia.org/wiki/T-statistic

3 Calcular o p-value através de stats.t.sf

4 Rejeiar ou não a hipótese usando uma significância de 0.10

In [339]:
# degrees of freedom
df["df"] = df[["ab_n", "c_n"]].min(axis=1)
df["df"] = df["df"] - 1
df["t_stat"] = (df.ab_mu - df.c_mu) / ((df.ab_std**2 / df.ab_n) + (df.c_std**2 / df.c_n))
df["p_value"] = stats.t.sf(np.abs(df.t_stat), df["df"])*2
df[df["p_value"] <= 0.10]

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n,df,t_stat,p_value
,Calc_Social,Calc_Social,Calc_Social,Calc_Social,Calc_Social,Calc_Social,,,
Celebridade,,,,,,,,,
Carol,29.006579,10.192858,304,27.510563,10.520545,284,283,2.045183,0.041762
Gilda,32.422581,8.681648,310,30.886525,10.186931,282,281,2.513494,0.012514
Luana,30.919014,9.434007,284,29.700000,10.084316,260,259,1.730299,0.084768
Luciana,29.504043,10.395782,371,28.232558,11.714339,301,300,1.701668,0.089854
Marina,30.361290,9.594958,310,28.717131,10.262342,251,250,2.294509,0.022591


### Resultado: Apelo Social entre as Classes

Verifica-se que muitas celebridades possuem apelos diferentes entre as classes. Notorialmente a Carol e a Gilda, que novamente atuam como fator de distinção entre os grupos.

Obs: Apesar da segmentaço, as duas apresentam notas altas, entretanto, para entender essa variação seria necessário uma avaliação mais cuidadosa do perfil de cada uma.

# Apelo Comercial
### Variável Agregada

**A seguir defini as features que, segundo meu critério (especialista seria o ideal), estão relacionadas com o apelo comercial da celebridade visto pelo entrevistado.**

In [340]:
features_apelo_comercial = [
    "Cel_TenhoVisto",
    "Cel_PerfilPropagandas",
    "Cel_EuCompraria"
]

In [341]:
cls_c["Calc_Comercial"] = 0
cls_ab["Calc_Comercial"] = 0

for feature in features_apelo_comercial:
    cls_c["Calc_Comercial"] += cls_c[feature]
    cls_ab["Calc_Comercial"] += cls_ab[feature]

cls_c["Calc_Comercial"] = pd.to_numeric(cls_c["Calc_Comercial"])
cls_ab["Calc_Comercial"] = pd.to_numeric(cls_ab["Calc_Comercial"])

ab_mu = cls_ab[["Celebridade", "Calc_Comercial"]].groupby("Celebridade").mean()
c_mu = cls_c[["Celebridade", "Calc_Comercial"]].groupby("Celebridade").mean()

ab_std = cls_ab[["Celebridade", "Calc_Comercial"]].groupby("Celebridade").std()
c_std = cls_c[["Celebridade", "Calc_Comercial"]].groupby("Celebridade").std()

ab_n = cls_ab[["Celebridade", "Calc_Comercial"]].groupby("Celebridade").count()
c_n = cls_c[["Celebridade", "Calc_Comercial"]].groupby("Celebridade").count()

df = pd.concat([ab_mu, ab_std, ab_n, c_mu, c_std, c_n], axis=1, 
               keys=["ab_mu", "ab_std", "ab_n", "c_mu", "c_std", "c_n"])
df

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n
,Calc_Comercial,Calc_Comercial,Calc_Comercial,Calc_Comercial,Calc_Comercial,Calc_Comercial
Celebridade,,,,,,
Beatriz,15.212308,4.927985,325,14.988189,5.519983,254
Bruna,16.033951,4.350235,324,15.631579,4.863377,266
Carol,16.430921,4.409167,304,15.644366,4.681775,284
Clara,15.099415,5.201686,342,14.908425,4.947040,273
Fátima,15.797468,4.858871,395,16.011905,4.950487,336
Gilda,17.219355,3.987474,310,16.734043,4.441415,282
Iris,17.348485,4.172906,330,17.340741,4.118438,270
Juliana,16.677812,4.518879,329,16.775362,3.917361,276


## Hipótese 5: Há diferença de apelo comercial entre as celebridades e as classes sociais AB e C+?

**Teste de Hipótese entre as médias das classes AB e C+**

* Ho: mu_ab - mu_c+ = 0
* Ha: mu_ab - mu_c+ != 0

O objetivo é selecionar celebridades que não possuem diferença de apelo comercial, ou seja, o p-value > 0.10

**Framework**

1 Calcular os degrees of freedom para distribuição t

df = n - 1

2 Calcultar a t-statistic para cada celebridade

https://en.m.wikipedia.org/wiki/T-statistic

3 Calcular o p-value através de stats.t.sf

4 Rejeiar ou não a hipótese usando uma significância de 0.10

In [342]:
# degrees of freedom
df["df"] = df[["ab_n", "c_n"]].min(axis=1)
df["df"] = df["df"] - 1
df["t_stat"] = (df.ab_mu - df.c_mu) / ((df.ab_std**2 / df.ab_n) + (df.c_std**2 / df.c_n))
df["p_value"] = stats.t.sf(np.abs(df.t_stat), df["df"])*2
df[df["p_value"] <= 0.10]

,ab_mu,ab_std,ab_n,c_mu,c_std,c_n,df,t_stat,p_value
,Calc_Comercial,Calc_Comercial,Calc_Comercial,Calc_Comercial,Calc_Comercial,Calc_Comercial,,,
Celebridade,,,,,,,,,
Bruna,16.033951,4.350235,324,15.631579,4.863377,266,265,2.731128,6.735508e-03
Carol,16.430921,4.409167,304,15.644366,4.681775,284,283,5.573285,5.825514e-08
Gilda,17.219355,3.987474,310,16.734043,4.441415,282,281,4.002869,8.013016e-05
Luana,16.193662,4.446918,284,15.361538,4.718327,260,259,5.359687,1.844185e-07
Luciana,15.964960,4.582736,371,15.225914,5.223868,301,300,5.018368,8.945798e-07
Maria,14.812139,4.918698,346,14.069444,5.244081,288,287,4.489990,1.032824e-05
Telma,17.093851,4.068520,309,16.656000,4.536517,250,249,3.222123,1.442126e-03


### Resultado: Apelo Comercial entre as Classes

Verifica-se que muitas celebridades possuem apelos diferentes entre as classes. Entretanto, a Gilda e Telma se destacam por possuirem

Obs: Apesar da segmentaçao, as duas apresentam notas altas, entretanto, para entender essa variação seria necessário uma avaliação mais cuidadosa do perfil de cada uma.

# Conclusão

Após extensa exploração dos dados e validação de algumas hipóteses avalio 
que a celebridade que melhor atende aos desejos do cliente é a chamada **Gilda**.
Pois ela, segundo o teste de proporção entre mulheres no geral e o público alvo
de meia-idade, possui uma taxa de reconhecimento maior. Além disso, 
quando analisado por classe social, com o objetivo de impactar mais pessoas 
das classes C+, ela possui resultados altos em ambos grupos, apesar de segregá-los.
